In [1]:
# Cell 1: Imports and Setup
import os
import re # For parsing filenames more robustly
import cv2
import numpy as np
import torch
import matplotlib.pyplot as plt
from pathlib import Path
import sys
import math

# Add SuperGlue directory to Python path
superglue_path = Path('SuperGluePretrainedNetwork')
sys.path.append(str(superglue_path))

# Import SuperGlue model components (adjust based on actual file structure if needed)
from models.matching import Matching
from models.utils import (frame2tensor, make_matching_plot_fast, AverageTimer, read_image)

# Setup device (GPU if available, otherwise CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# SuperGlue configuration (using outdoor weights as they are generally robust)
config = {
    'superpoint': {
        'nms_radius': 4,
        'keypoint_threshold': 0.005,
        'max_keypoints': -1 # Use -1 for no limit, or set a number like 1024 or 2048
    },
    'superglue': {
        'weights': 'outdoor', # 'indoor' or 'outdoor'
        'sinkhorn_iterations': 20,
        'match_threshold': 0.2,
    }
}

# --- Constants ---
QUERY_DIR = Path('./extended_dataset/camera image')
REF_DIR = Path('./extended_dataset/reference image')
REFERENCE_IMAGE_SIZE = (1000, 1000) # Assuming all reference images are 1000x1000
VIZ_COUNT = 5 # Number of results to visualize

Using device: cuda


In [2]:
# Cell 2: Function to Parse Filename
def parse_filename(filename):
    """
    Parses the filename to extract ground truth coordinates and angle.

    Filename format: (ignore)_(x,y)_angle_(ignore).png
    Coordinates: (x, y) relative to reference center. +x down, +y right.
    Angle: Anti-clockwise rotation degrees (0-360).
    """
    try:
        parts = filename.stem.split('_') # Use stem to remove .png
        if len(parts) < 3:
            print(f"Warning: Skipping poorly formatted file: {filename.name}")
            return None

        # Robustly find the coordinate tuple part like '(float,float)'
        coord_match = re.search(r'\((-?\d+\.?\d*),(-?\d+\.?\d*)\)', parts[1])
        if not coord_match:
            # Try finding it in the second part if the first was just coords like '507,4801'
             coord_match = re.search(r'\((-?\d+\.?\d*),(-?\d+\.?\d*)\)', parts[1] if len(parts) > 1 else '')
             if not coord_match and len(parts) > 2:
                 coord_match = re.search(r'\((-?\d+\.?\d*),(-?\d+\.?\d*)\)', parts[2])

        if not coord_match:
             # Handle case where coords might not have parentheses like '97.86,-113.64'
             coord_parts = parts[1].split(',')
             if len(coord_parts) == 2:
                 try:
                     gt_x = float(coord_parts[0])
                     gt_y = float(coord_parts[1])
                 except ValueError:
                     print(f"Warning: Could not parse coordinates from {parts[1]} in file: {filename.name}")
                     return None
             else: # Try the next part
                 if len(parts) > 2:
                     coord_parts = parts[2].split(',')
                     if len(coord_parts) == 2:
                         try:
                             gt_x = float(coord_parts[0])
                             gt_y = float(coord_parts[1])
                         except ValueError:
                             print(f"Warning: Could not parse coordinates from {parts[2]} in file: {filename.name}")
                             return None
                     else:
                          print(f"Warning: Could not find coordinate tuple in file: {filename.name}")
                          return None
                 else:
                      print(f"Warning: Could not find coordinate tuple in file: {filename.name}")
                      return None

        else:
            gt_x = float(coord_match.group(1))
            gt_y = float(coord_match.group(2))


        # Find the angle part (float) - usually the part after coordinates
        angle_part_index = -1
        for i in range(1, len(parts) -1): # Check parts between ignore and location
             # Check if it looks like the coordinate tuple we just found
            is_coord_part = False
            if coord_match:
                 if parts[i] == f"({coord_match.group(1)},{coord_match.group(2)})":
                     is_coord_part = True
            elif len(parts[i].split(',')) == 2: # Check if it matches the non-parenthesis coords
                 try:
                     float(parts[i].split(',')[0])
                     float(parts[i].split(',')[1])
                     is_coord_part = True
                 except ValueError:
                     pass

            if is_coord_part and i + 1 < len(parts) -1 : # If this was the coord part, angle should be next
                 try:
                     gt_angle = float(parts[i+1])
                     angle_part_index = i+1
                     break
                 except ValueError:
                     pass # Continue searching

        # If not found after coordinates, try finding the first float number after the first underscore
        if angle_part_index == -1:
             for i in range(1, len(parts) - 1):
                  try:
                      gt_angle = float(parts[i])
                      # Avoid accidentally parsing the first coordinate if it's simple like '(500)'
                      if not parts[i].startswith('(') or not parts[i].endswith(')'):
                          angle_part_index = i
                          break
                  except ValueError:
                     continue # Not a float, try next part

        if angle_part_index == -1:
             print(f"Warning: Could not find angle value in file: {filename.name}")
             return None


        # Validate coordinate ranges (optional but good practice)
        # if not (-500 <= gt_x <= 500 and -500 <= gt_y <= 500):
        #     print(f"Warning: Coordinates out of range [-500, 500] in file: {filename.name}")
            # return None # Or just continue if you want to process them anyway

        # Validate angle range
        if not (0 <= gt_angle <= 360):
             # Allow for slight floating point inaccuracies near 0/360
             if not (abs(gt_angle) < 1e-9 or abs(gt_angle - 360) < 1e-9):
                 print(f"Warning: Angle out of range [0, 360] in file: {filename.name}")
                 # Handle wrap around? e.g., 360.1 -> 0.1? or -0.1 -> 359.9?
                 gt_angle = gt_angle % 360
                 print(f"         Adjusted angle to {gt_angle:.2f}")
                 #return None # Or adjust angle: gt_angle = gt_angle % 360

        return {'x': gt_x, 'y': gt_y, 'angle': gt_angle}

    except Exception as e:
        print(f"Error parsing filename {filename.name}: {e}")
        return None

In [3]:
# Cell 3: Function to Estimate Transformation
def estimate_transform(kp0, kp1, matches, confidence):
    """
    Estimates rotation and translation from keypoint matches using RANSAC.
    Returns:
        - M: 2x3 Affine transformation matrix (or None if failed)
        - angle_deg: Estimated rotation in degrees (anti-clockwise, 0-360)
        - trans_px: Estimated translation (tx, ty) in pixels (relative to top-left)
        - mask: RANSAC inlier mask
    """
    mkpts0 = kp0[matches[:, 0]]
    mkpts1 = kp1[matches[:, 1]]

    # Filter based on confidence
    min_req_matches = 4 # Minimum matches for estimateAffine2D
    indices = np.where(confidence > config['superglue']['match_threshold'])[0]
    mkpts0 = mkpts0[indices]
    mkpts1 = mkpts1[indices]

    if mkpts0.shape[0] < min_req_matches:
        # print(f"Not enough confident matches ({mkpts0.shape[0]})")
        return None, None, None, None

    # Use cv2.estimateAffine2D for rotation, translation, and scale
    # It requires at least 3 points, RANSAC helps with outliers.
    M, mask = cv2.estimateAffine2D(mkpts0, mkpts1, method=cv2.RANSAC,
                                     ransacReprojThreshold=5.0) # Adjust threshold as needed

    if M is None or mask is None:
        # print("RANSAC failed to find a transformation.")
        return None, None, None, None

    # Check number of inliers
    num_inliers = np.sum(mask)
    if num_inliers < min_req_matches:
        # print(f"Not enough RANSAC inliers ({num_inliers})")
        return None, None, None, None


    # Decompose the Affine matrix M = [[a, b, tx], [c, d, ty]]
    # Assuming M = [[s*cos(theta), -s*sin(theta), tx], [s*sin(theta), s*cos(theta), ty]]
    # where s is scale and theta is the rotation angle

    # Extract translation
    tx = M[0, 2]
    ty = M[1, 2]
    trans_px = (tx, ty)

    # Extract rotation angle
    # angle_rad = atan2(c, a) or atan2(-b, d) - use atan2(M[1,0], M[0,0])
    angle_rad = math.atan2(M[1, 0], M[0, 0])
    angle_deg_raw = np.degrees(angle_rad)

    # Convert OpenCV angle (rotation of coordinate system) to anti-clockwise image rotation (0-360)
    # A positive angle_rad from atan2(M[1,0], M[0,0]) corresponds to an anti-clockwise rotation
    # of the coordinate axes. To get the image rotation (how much the object turned anti-clockwise),
    # we usually take the negative, but let's verify convention.
    # If query is rotated A degrees anti-clockwise relative to ref:
    # Point p_q in query becomes p_r in reference.
    # p_r = RotationMatrix(A) * p_q + Translation
    # OpenCV's M transforms query points *to* reference points.
    # So M[0,0] = s*cos(A), M[1,0] = s*sin(A).
    # angle_rad = atan2(M[1,0], M[0,0]) = atan2(s*sin(A), s*cos(A)) = A
    # So, the angle A is directly angle_rad. We need it in 0-360 range.
    angle_deg = angle_deg_raw % 360
    if angle_deg < 0:
         angle_deg += 360 # Ensure positive range [0, 360)

    # Optional: Extract scale (should be close to 1 if images are same scale)
    # scale_x = np.sqrt(M[0, 0]**2 + M[1, 0]**2)
    # scale_y = np.sqrt(M[0, 1]**2 + M[1, 1]**2)
    # print(f"Estimated scale: sx={scale_x:.2f}, sy={scale_y:.2f}")

    return M, angle_deg, trans_px, mask

In [4]:
# Cell 4: Function for Coordinate Conversion
def convert_to_relative_center_coords(trans_px_tl, angle_deg, query_shape, ref_shape):
    """
    Converts the translation relative to top-left corners and rotation
    to the translation relative to image centers, using the user's convention.

    Args:
        trans_px_tl (tuple): (tx, ty) translation from query top-left to ref top-left (OpenCV standard).
        angle_deg (float): Anti-clockwise rotation angle in degrees.
        query_shape (tuple): (height, width) of the query image.
        ref_shape (tuple): (height, width) of the reference image.

    Returns:
        tuple: (rel_x, rel_y) relative center coordinates (+x down, +y right).
               Returns (None, None) if input is invalid.
    """
    if trans_px_tl is None or angle_deg is None:
        return None, None

    q_h, q_w = query_shape[:2]
    r_h, r_w = ref_shape[:2]

    # Reference image center (in OpenCV coords: origin top-left, +x right, +y down)
    ref_center_cv_x = r_w / 2.0
    ref_center_cv_y = r_h / 2.0

    # Query image center (in its own local coords: origin top-left)
    query_center_local_x = q_w / 2.0
    query_center_local_y = q_h / 2.0

    # Calculate where the query center ends up in the reference image coordinate system (origin top-left)
    # We need the full affine matrix M for this, as simple translation isn't enough due to rotation.
    # Let's re-calculate M using the estimated components (assuming scale=1)
    angle_rad = np.radians(angle_deg)
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)
    tx, ty = trans_px_tl

    M = np.array([
        [cos_a, -sin_a, tx],
        [sin_a,  cos_a, ty]
    ])

    # Transform the query center using the affine matrix M
    # ref_coords = M @ [query_local_x, query_local_y, 1]
    query_center_in_ref_x = M[0, 0] * query_center_local_x + M[0, 1] * query_center_local_y + M[0, 2]
    query_center_in_ref_y = M[1, 0] * query_center_local_x + M[1, 1] * query_center_local_y + M[1, 2]

    # Now, find the vector from the reference center to the query center (in OpenCV coords)
    vec_cv_x = query_center_in_ref_x - ref_center_cv_x
    vec_cv_y = query_center_in_ref_y - ref_center_cv_y

    # Convert this vector to the user's coordinate system (+x down, +y right)
    rel_x = vec_cv_y # User's x is OpenCV's y
    rel_y = vec_cv_x # User's y is OpenCV's x

    return rel_x, rel_y

In [5]:
# Cell 5: Load Model
print("Loading SuperGlue model...")
matching = Matching(config).eval().to(device)
print("Model loaded.")

Loading SuperGlue model...
Loaded SuperPoint model


/home/jain/Desktop/IITK/Project_EE798T/SuperGluePretrainedNetwork/models/superpoint.py:137: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch.load(str

Loaded SuperGlue model ("outdoor" weights)
Model loaded.


In [6]:
# Cell 6: Main Processing Loop
results = []
all_filenames = []

# Ensure directories exist
if not QUERY_DIR.is_dir() or not REF_DIR.is_dir():
    raise FileNotFoundError("Query or Reference image directory not found.")

# Get list of query images, assuming reference images have the same names
# Ignore hidden files like .DS_Store
query_files = sorted([f for f in QUERY_DIR.iterdir() if f.is_file() and not f.name.startswith('.') and f.suffix.lower() == '.png'])

if not query_files:
    print(f"No PNG files found in {QUERY_DIR}")
else:
    print(f"Found {len(query_files)} query images. Processing...")

for query_path in query_files:
    filename = query_path.name
    ref_path = REF_DIR / filename

    if not ref_path.exists():
        print(f"Warning: Reference image not found for {filename}. Skipping.")
        continue

    # 1. Parse Ground Truth
    ground_truth = parse_filename(query_path)
    if ground_truth is None:
        continue # Skip if parsing failed

    try:
        # 2. Load Images (using read_image from SuperGlue utils for consistency)
        # read_image loads in grayscale and converts to float tensor [0,1]
        image0, inp0, scales0 = read_image(str(query_path), device, [-1], 0, False) # Query
        image1, inp1, scales1 = read_image(str(ref_path), device, [-1], 0, False)   # Reference

        if image0 is None or image1 is None:
             print(f"Warning: Could not load images for {filename}. Skipping.")
             continue

        query_h, query_w = image0.shape[:2]
        ref_h, ref_w = image1.shape[:2]

        # Check if reference image size is as expected
        if ref_h != REFERENCE_IMAGE_SIZE[0] or ref_w != REFERENCE_IMAGE_SIZE[1]:
            print(f"Warning: Reference image {filename} has unexpected size {image1.shape[:2]}. Expected {REFERENCE_IMAGE_SIZE}. Adjusting calculations.")
            # You might need to resize or handle this differently if sizes vary significantly

        # 3. Perform Matching
        with torch.no_grad():
            pred = matching({'image0': inp0, 'image1': inp1})

        # Detach tensors and move to CPU for NumPy/OpenCV processing
        pred = {k: v[0].cpu().numpy() for k, v in pred.items()}
        kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
        matches, conf = pred['matches0'], pred['matching_scores0']

        # Keep only valid matches (index != -1)
        valid = matches > -1
        mkpts0_idx = np.where(valid)[0]
        mkpts1_idx = matches[valid]
        conf = conf[valid]
        # Create the match array format expected by estimate_transform: Nx2 array of indices
        match_indices = np.stack([mkpts0_idx, mkpts1_idx], axis=1)

        # 4. Estimate Transformation
        M, pred_angle, trans_px_tl, _ = estimate_transform(kpts0, kpts1, match_indices, conf)

        # 5. Convert Coordinates
        if M is not None:
            pred_x, pred_y = convert_to_relative_center_coords(trans_px_tl, pred_angle, (query_h, query_w), (ref_h, ref_w))
        else:
            pred_x, pred_y, pred_angle = None, None, None # Indicate failure

        # 6. Store Results
        results.append({
            'filename': filename,
            'query_path': str(query_path),
            'ref_path': str(ref_path),
            'gt_x': ground_truth['x'],
            'gt_y': ground_truth['y'],
            'gt_angle': ground_truth['angle'],
            'pred_x': pred_x,
            'pred_y': pred_y,
            'pred_angle': pred_angle,
            'query_shape': (query_h, query_w),
            'ref_shape': (ref_h, ref_w),
            # Store intermediate results for visualization if needed
            'kpts0': kpts0,
            'kpts1': kpts1,
            'matches': match_indices,
            'confidence': conf
        })
        all_filenames.append(filename) # Keep track even if prediction failed

    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        # Optionally store failure information
        results.append({
            'filename': filename,
            'query_path': str(query_path),
            'ref_path': str(ref_path),
            'gt_x': ground_truth.get('x'), # Use .get in case parsing failed earlier
            'gt_y': ground_truth.get('y'),
            'gt_angle': ground_truth.get('angle'),
            'pred_x': None,
            'pred_y': None,
            'pred_angle': None,
            'error': str(e)
        })
        all_filenames.append(filename)


print(f"\nProcessed {len(results)} out of {len(query_files)} files.")
# If you wanted train/test split, you could use sklearn here on 'all_filenames'
# from sklearn.model_selection import train_test_split
# train_files, test_files = train_test_split(all_filenames, test_size=0.2, random_state=42)
# Then filter the 'results' list based on these filenames.
# For now, we evaluate on all processed files.

Found 2040 query images. Processing...

Processed 2040 out of 2040 files.


In [7]:
# Cell 7: Calculate Accuracy Metrics
coord_errors = []
angle_errors = []
successful_predictions = 0

for res in results:
    if res['pred_x'] is not None and res['pred_y'] is not None and res['pred_angle'] is not None:
        successful_predictions += 1

        # Coordinate Error (Euclidean Distance)
        dx = res['pred_x'] - res['gt_x']
        dy = res['pred_y'] - res['gt_y']
        coord_err = np.sqrt(dx**2 + dy**2)
        coord_errors.append(coord_err)

        # Angle Error (handle wrap-around)
        angle_diff = abs(res['pred_angle'] - res['gt_angle'])
        angle_err = min(angle_diff, 360.0 - angle_diff)
        angle_errors.append(angle_err)

print(f"\n--- Accuracy Metrics ({successful_predictions} successful predictions) ---")
if successful_predictions > 0:
    mean_coord_error = np.mean(coord_errors)
    median_coord_error = np.median(coord_errors)
    std_coord_error = np.std(coord_errors)

    mean_angle_error = np.mean(angle_errors)
    median_angle_error = np.median(angle_errors)
    std_angle_error = np.std(angle_errors)

    print(f"Coordinate Error (pixels):")
    print(f"  Mean:   {mean_coord_error:.3f}")
    print(f"  Median: {median_coord_error:.3f}")
    print(f"  StdDev: {std_coord_error:.3f}")

    print(f"\nAngle Error (degrees):")
    print(f"  Mean:   {mean_angle_error:.3f}")
    print(f"  Median: {median_angle_error:.3f}")
    print(f"  StdDev: {std_angle_error:.3f}")

    # Optional: Print quantiles or histogram for better error distribution understanding
    # print(f"\nCoord Error Quantiles (px): {np.quantile(coord_errors, [0.25, 0.5, 0.75, 0.9, 0.95])}")
    # print(f"Angle Error Quantiles (deg): {np.quantile(angle_errors, [0.25, 0.5, 0.75, 0.9, 0.95])}")

    # plt.figure(figsize=(12, 5))
    # plt.subplot(1, 2, 1)
    # plt.hist(coord_errors, bins=50)
    # plt.title('Coordinate Error Distribution')
    # plt.xlabel('Error (pixels)')
    # plt.ylabel('Frequency')
    # plt.subplot(1, 2, 2)
    # plt.hist(angle_errors, bins=50)
    # plt.title('Angle Error Distribution')
    # plt.xlabel('Error (degrees)')
    # plt.ylabel('Frequency')
    # plt.tight_layout()
    # plt.show()

else:
    print("No successful predictions were made.")

print(f"Total files processed: {len(results)}")
print(f"Prediction failure rate: {(len(results) - successful_predictions) / len(results) * 100 if len(results) > 0 else 0:.2f}%")


--- Accuracy Metrics (867 successful predictions) ---
Coordinate Error (pixels):
  Mean:   33.953
  Median: 0.806
  StdDev: 226.254

Angle Error (degrees):
  Mean:   8.282
  Median: 0.034
  StdDev: 33.833
Total files processed: 2040
Prediction failure rate: 57.50%


In [8]:
# Cell 8: Visualization Function
def get_rotated_rect_corners(center_x, center_y, w, h, angle_deg, ref_shape):
    """
    Calculates the four corner points of a rectangle rotated around its center.
    Center coordinates are relative to the reference image center (+x down, +y right).
    Angle is anti-clockwise degrees.
    Returns corners in OpenCV format (integer tuples, origin top-left).
    """
    r_h, r_w = ref_shape[:2]
    ref_center_cv_x = r_w / 2.0
    ref_center_cv_y = r_h / 2.0

    # Convert user center coords back to OpenCV top-left origin coords
    center_cv_x = center_y + ref_center_cv_x # User y is CV x
    center_cv_y = center_x + ref_center_cv_y # User x is CV y

    # Angle for rotation matrix (negative for rotating points)
    angle_rad = np.radians(-angle_deg) # Need to rotate points clockwise if angle is anti-clockwise image rotation
    cos_a = np.cos(angle_rad)
    sin_a = np.sin(angle_rad)

    # Rectangle corners relative to its center (0,0)
    hw = w / 2.0
    hh = h / 2.0
    corners_local = np.array([
        [-hw, -hh], # Top-left
        [ hw, -hh], # Top-right
        [ hw,  hh], # Bottom-right
        [-hw,  hh]  # Bottom-left
    ])

    # Rotate and translate corners
    corners_rotated = np.zeros_like(corners_local)
    for i, (x, y) in enumerate(corners_local):
        x_rot = x * cos_a - y * sin_a
        y_rot = x * sin_a + y * cos_a
        corners_rotated[i, 0] = x_rot + center_cv_x
        corners_rotated[i, 1] = y_rot + center_cv_y

    # Return as integer tuples
    return corners_rotated.astype(int)


def visualize_result(result_data, border_thickness=3):
    """
    Visualizes the query, reference, and localization boxes (predicted vs ground truth).
    """
    query_img_bgr = cv2.imread(result_data['query_path'])
    ref_img_bgr = cv2.imread(result_data['ref_path'])

    if query_img_bgr is None or ref_img_bgr is None:
        print(f"Error loading images for visualization: {result_data['filename']}")
        return

    q_h, q_w = result_data['query_shape'][:2]
    ref_shape = result_data['ref_shape']

    # Draw Ground Truth Box (Green)
    gt_corners = get_rotated_rect_corners(
        result_data['gt_x'], result_data['gt_y'],
        q_w, q_h, result_data['gt_angle'], ref_shape
    )
    cv2.polylines(ref_img_bgr, [gt_corners], isClosed=True, color=(0, 255, 0), thickness=border_thickness) # Green

    # Draw Predicted Box (Red) - only if prediction was successful
    if result_data['pred_x'] is not None:
        pred_corners = get_rotated_rect_corners(
            result_data['pred_x'], result_data['pred_y'],
            q_w, q_h, result_data['pred_angle'], ref_shape
        )
        cv2.polylines(ref_img_bgr, [pred_corners], isClosed=True, color=(0, 0, 255), thickness=border_thickness) # Red
        pred_status = "Successful"
        coord_err = np.sqrt((result_data['pred_x'] - result_data['gt_x'])**2 + (result_data['pred_y'] - result_data['gt_y'])**2)
        angle_diff = abs(result_data['pred_angle'] - result_data['gt_angle'])
        angle_err = min(angle_diff, 360.0 - angle_diff)
        error_text = f"Coord Err: {coord_err:.2f}px, Angle Err: {angle_err:.2f}deg"
    else:
        pred_status = "Failed"
        error_text = "Prediction Failed"

    # --- Display using Matplotlib ---
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))

    # Display Query Image
    axes[0].imshow(cv2.cvtColor(query_img_bgr, cv2.COLOR_BGR2RGB))
    axes[0].set_title(f"Query: {result_data['filename']}")
    axes[0].axis('off')

    # Display Reference Image with Boxes
    axes[1].imshow(cv2.cvtColor(ref_img_bgr, cv2.COLOR_BGR2RGB))
    axes[1].set_title(f"Reference (GT=Green, Pred=Red) - {pred_status}")
    axes[1].text(5, 25, error_text, color='yellow', fontsize=10, bbox=dict(facecolor='black', alpha=0.5))
    axes[1].axis('off')

    plt.suptitle(f"Ground Truth: ({result_data['gt_x']:.2f}, {result_data['gt_y']:.2f}), {result_data['gt_angle']:.2f}°\n"
                 f"Prediction: ({result_data['pred_x']:.2f}, {result_data['pred_y']:.2f}), {result_data['pred_angle']:.2f}°" if pred_status=="Successful" else \
                 f"Ground Truth: ({result_data['gt_x']:.2f}, {result_data['gt_y']:.2f}), {result_data['gt_angle']:.2f}°\nPrediction: Failed", y=0.95)


    plt.tight_layout(rect=[0, 0.03, 1, 0.9]) # Adjust layout to prevent title overlap
    plt.show()

In [9]:
# Cell 8.5: Separate Results and Calculate Errors

successful_results = []
failed_results = []
high_error_results = [] # New list

# Define error thresholds
COORD_ERROR_THRESHOLD = 50.0 # pixels
ANGLE_ERROR_THRESHOLD = 20.0 # degrees

if 'results' in locals() and isinstance(results, list):
    for res in results:
        is_successful = (res.get('pred_x') is not None and
                         res.get('pred_y') is not None and
                         res.get('pred_angle') is not None)

        if is_successful:
            # Calculate errors for successful predictions
            dx = res['pred_x'] - res['gt_x']
            dy = res['pred_y'] - res['gt_y']
            coord_err = np.sqrt(dx**2 + dy**2)

            angle_diff = abs(res['pred_angle'] - res['gt_angle'])
            angle_err = min(angle_diff, 360.0 - angle_diff)

            # Store errors back into the dictionary
            res['coord_error'] = coord_err
            res['angle_error'] = angle_err

            successful_results.append(res) # Add to successful list

            # Check if it qualifies for high error list
            if coord_err > COORD_ERROR_THRESHOLD or angle_err > ANGLE_ERROR_THRESHOLD:
                high_error_results.append(res)

        else:
            # Add placeholder errors for failed results if needed later, otherwise just add to list
            res['coord_error'] = None
            res['angle_error'] = None
            failed_results.append(res)

    print(f"Separation and Error Calculation complete:")
    print(f"  Found {len(successful_results)} successful predictions.")
    print(f"  Found {len(failed_results)} failed predictions.")
    print(f"  Identified {len(high_error_results)} successful predictions with high error.")
    # Optional: Sort high_error_results if desired (e.g., by coordinate error descending)
    # high_error_results.sort(key=lambda x: x.get('coord_error', 0), reverse=True)

else:
    print("ERROR: 'results' variable not found or is not a list.")
    # Define empty lists to prevent errors later
    successful_results = []
    failed_results = []
    high_error_results = []

Separation and Error Calculation complete:
  Found 867 successful predictions.
  Found 1173 failed predictions.
  Identified 54 successful predictions with high error.


In [10]:
# Cell 8.7: Attempt to force inline backend before visualization
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
print("Attempted to switch backend to inline.")
# Optional: re-import display function from IPython if needed, though usually not required for inline
# from IPython.display import display

Attempted to switch backend to inline.


In [11]:
# Cell 9: Perform Visualization for Failed, High-Error Successful, and Generally Successful Examples

MAX_VIZ_EACH = 5 # Number of examples to show for each category

# --- Visualize FAILED Predictions ---
print(f"\n{'='*20} Visualizing up to {MAX_VIZ_EACH} FAILED Prediction Examples {'='*20}")
# (Keep the existing code for visualizing failed_results here...)
if 'failed_results' not in locals() or not isinstance(failed_results, list):
     print("ERROR: 'failed_results' list not found. Did you run the cell above (Cell 8.5)?")
elif not failed_results:
    print("\nNo failed prediction results found to visualize.")
else:
    # ... (visualization loop for failed_results) ...
    print(f"\nFinished displaying {count_failed} failed examples.") # Make sure count_failed is defined in the loop

# --- Visualize HIGH ERROR Successful Predictions ---
print(f"\n{'='*20} Visualizing up to {MAX_VIZ_EACH} HIGH ERROR Successful Prediction Examples {'='*20}")
print(f"(Thresholds: Coord > {COORD_ERROR_THRESHOLD}px OR Angle > {ANGLE_ERROR_THRESHOLD}deg)")

if 'high_error_results' not in locals() or not isinstance(high_error_results, list):
     print("ERROR: 'high_error_results' list not found. Did you run the cell above (Cell 8.5)?")
elif not high_error_results:
    print("\nNo successful predictions with high error found to visualize.")
else:
    print(f"\nDisplaying the first {min(len(high_error_results), MAX_VIZ_EACH)} high-error successful examples...")
    count_hierr = 0
    for res in high_error_results:
        if count_hierr < MAX_VIZ_EACH:
            # Display calculated errors clearly
            coord_err_str = f"{res.get('coord_error', 'N/A'):.2f}" if res.get('coord_error') is not None else "N/A"
            angle_err_str = f"{res.get('angle_error', 'N/A'):.2f}" if res.get('angle_error') is not None else "N/A"
            print(f"\n[{count_hierr+1}/{min(len(high_error_results), MAX_VIZ_EACH)}] Visualizing HIGH ERROR result for: {res.get('filename', 'N/A')}")
            print(f"  Errors - Coord: {coord_err_str}px, Angle: {angle_err_str}deg")
            try:
                visualize_result(res) # Assumes visualize_result is defined in Cell 8
                count_hierr += 1
            except Exception as e:
                 print(f"  ERROR during visualization for {res.get('filename', 'N/A')}: {e}")
        else:
            break # Stop after showing MAX_VIZ_EACH
    if count_hierr == 0 and high_error_results:
         print("\nCould not visualize any of the available high-error results due to errors during visualization.")
    print(f"\nFinished displaying {count_hierr} high-error successful examples.")

# --- Visualize GENERALLY Successful Predictions (Optional: filter out high errors) ---
# Option 1: Show first 5 overall successful (might include high errors again)
# Option 2: Show first 5 successful that are *not* in high_error_results (shows low errors)
# Let's go with Option 2 for clarity
print(f"\n{'='*20} Visualizing up to {MAX_VIZ_EACH} LOW/MEDIUM Error Successful Prediction Examples {'='*20}")

if 'successful_results' not in locals() or not isinstance(successful_results, list):
     print("ERROR: 'successful_results' list not found. Did you run the cell above (Cell 8.5)?")
elif not successful_results:
     print("\nNo successful predictions found at all.")
else:
    # Filter out high-error examples already shown
    low_med_error_results = [res for res in successful_results if res not in high_error_results]

    if not low_med_error_results:
         print("\nNo successful predictions with low/medium error found (all successful had high error or failed visualization).")
    else:
        print(f"\nDisplaying the first {min(len(low_med_error_results), MAX_VIZ_EACH)} low/medium-error successful examples...")
        count_success = 0
        for res in low_med_error_results:
            if count_success < MAX_VIZ_EACH:
                coord_err_str = f"{res.get('coord_error', 'N/A'):.2f}" if res.get('coord_error') is not None else "N/A"
                angle_err_str = f"{res.get('angle_error', 'N/A'):.2f}" if res.get('angle_error') is not None else "N/A"
                print(f"\n[{count_success+1}/{min(len(low_med_error_results), MAX_VIZ_EACH)}] Visualizing LOW/MED ERROR result for: {res.get('filename', 'N/A')}")
                print(f"  Errors - Coord: {coord_err_str}px, Angle: {angle_err_str}deg")
                try:
                    visualize_result(res) # Assumes visualize_result is defined in Cell 8
                    count_success += 1
                except Exception as e:
                     print(f"  ERROR during visualization for {res.get('filename', 'N/A')}: {e}")
            else:
                break # Stop after showing MAX_VIZ_EACH
        if count_success == 0 and low_med_error_results:
             print("\nCould not visualize any of the available low/medium-error results due to errors during visualization.")
        print(f"\nFinished displaying {count_success} low/medium-error successful examples.")

print(f"\n{'='*60}")
print("Visualization complete.")


==================== Visualizing up to 5 FAILED Prediction Examples ====================


NameError: name 'count_failed' is not defined

In [12]:
# Cell 9.7: Checking Functions are loaded
print("--- Checking Definitions ---")
print(f"'matching' defined: {'matching' in locals()}")
print(f"'config' defined: {'config' in locals()}")
print(f"'device' defined: {'device' in locals()}")
print(f"'QUERY_DIR' defined: {'QUERY_DIR' in locals()}")
print(f"'REF_DIR' defined: {'REF_DIR' in locals()}")
print(f"'read_image' defined: {'read_image' in locals()}")
print(f"'estimate_transform' defined: {'estimate_transform' in locals()}")
print(f"'convert_to_relative_center_coords' defined: {'convert_to_relative_center_coords' in locals()}")
print("---------------------------")

--- Checking Definitions ---
'matching' defined: True
'config' defined: True
'device' defined: True
'QUERY_DIR' defined: True
'REF_DIR' defined: True
'read_image' defined: True
'estimate_transform' defined: True
'convert_to_relative_center_coords' defined: True
---------------------------


In [13]:
# Cell 10: Time Prediction for Random Sample (Corrected Version)

import time
import random
import numpy as np
import torch
import cv2 # Ensure OpenCV is imported
import math # Ensure math is imported
from pathlib import Path
import inspect # For checking functions/callables
import traceback # For detailed error printing

# --- Configuration ---
NUM_SAMPLES_FOR_TIMING = 5
# Make sure these paths are correct for your setup and match Cell 1
QUERY_DIR = Path('./extended_dataset/camera image')
REF_DIR = Path('./extended_dataset/reference image')

print(f"\n--- Timing Prediction for {NUM_SAMPLES_FOR_TIMING} Random Image Pairs ---")

# --- Pre-requisite Checks ---
# Check essential variables/functions defined in previous cells are available
required_vars = ['matching', 'config', 'device', 'QUERY_DIR', 'REF_DIR']
required_funcs = ['read_image', 'estimate_transform', 'convert_to_relative_center_coords'] # Note: estimate_transform isn't directly used below, but convert_... is
all_required = required_vars + required_funcs

missing_items = []
for item_name in all_required:
    if item_name not in locals():
         missing_items.append(item_name)

if missing_items:
    print("\nERROR: Missing required variables or functions.")
    missing_vars_found = [v for v in required_vars if v in missing_items]
    missing_funcs_found = [f for f in required_funcs if f in missing_items]
    if missing_vars_found:
         print(f"  Missing variables: {', '.join(missing_vars_found)}")
         print("  Please ensure Cells 1 and 5 were run successfully IN THIS KERNEL SESSION.")
    if missing_funcs_found:
         print(f"  Missing functions: {', '.join(missing_funcs_found)}")
         print("  Please ensure Cells defining these functions (SuperGlue utils, Cell 3, Cell 4) were run successfully IN THIS KERNEL SESSION.")
    print("\nCannot proceed with timing.")

else:
    # --- All checks passed, proceed with timing logic ---
    print("\n--- DEBUG: Pre-requisite checks passed. Proceeding with timing logic ---")
    # Use the variables confirmed to be in the local scope
    matching = locals()['matching']
    config = locals()['config']
    device = locals()['device']
    read_image = locals()['read_image']
    # estimate_transform = locals()['estimate_transform'] # Not directly used below
    convert_to_relative_center_coords = locals()['convert_to_relative_center_coords']

    try:
        # 1. Get list of valid image pairs
        if not QUERY_DIR.is_dir() or not REF_DIR.is_dir():
             raise FileNotFoundError(f"Query ({QUERY_DIR}) or Reference ({REF_DIR}) image directory not found.")

        all_query_files = sorted([
            f for f in QUERY_DIR.iterdir()
            if f.is_file() and not f.name.startswith('.') and f.suffix.lower() == '.png'
        ])

        valid_pairs = []
        for q_path in all_query_files:
            r_path = REF_DIR / q_path.name
            if r_path.exists():
                valid_pairs.append(q_path)

        if not valid_pairs:
             print("No valid image pairs found in the dataset to perform timing.")
             num_to_process = 0
        elif len(valid_pairs) < NUM_SAMPLES_FOR_TIMING:
            print(f"Warning: Only found {len(valid_pairs)} valid image pairs, "
                  f"which is less than the requested {NUM_SAMPLES_FOR_TIMING}.")
            num_to_process = len(valid_pairs)
        else:
             num_to_process = NUM_SAMPLES_FOR_TIMING

        if num_to_process > 0:
            # 2. Select random sample
            # random.seed(42) # Optional: for reproducible samples
            selected_files = random.sample(valid_pairs, num_to_process)
            print(f"Selected {num_to_process} files for timing:")
            for f in selected_files:
                print(f"  - {f.name}")

            # 3. Run predictions and time
            start_time = time.time() # Overall wall time start
            processed_count = 0
            prediction_times = [] # Store individual successful pair times

            for query_path in selected_files:
                filename = query_path.name
                ref_path = REF_DIR / filename
                pair_start_time = time.time() # Time each pair attempt

                try:
                    # --- Core Prediction Logic (Corrected Filtering) ---
                    # Load Images
                    image0, inp0, scales0 = read_image(str(query_path), device, [-1], 0, False) # Query
                    image1, inp1, scales1 = read_image(str(ref_path), device, [-1], 0, False)   # Reference
                    if image0 is None or image1 is None:
                        print(f"  Skipping {filename}: Image loading error.")
                        continue # Skip this pair
                    query_h, query_w = image0.shape[:2]
                    ref_h, ref_w = image1.shape[:2]

                    # Perform Matching
                    with torch.no_grad():
                        pred = matching({'image0': inp0, 'image1': inp1})
                    pred = {k: v[0].cpu().numpy() for k, v in pred.items()}
                    kpts0, kpts1 = pred['keypoints0'], pred['keypoints1']
                    matches, conf = pred['matches0'], pred['matching_scores0']

                    # ** CORRECTED FILTERING **
                    # Find indices where the match is valid AND confidence is above threshold
                    good_match_indices = np.where((matches > -1) & (conf > config['superglue']['match_threshold']))[0]

                    min_req_points = 3 # Minimum points for estimateAffine2D
                    if len(good_match_indices) < min_req_points:
                        print(f"  Skipping {filename}: Not enough confident matches found ({len(good_match_indices)} < {min_req_points}).")
                        continue # Skip this pair

                    # Get the coordinates of the keypoints for these good matches
                    mkpts0_coords = kpts0[good_match_indices]       # Coords from image 0
                    mkpts1_coords = kpts1[matches[good_match_indices]] # Coords from image 1

                    # Estimate Transformation directly using the filtered coordinates
                    M, mask = cv2.estimateAffine2D(mkpts0_coords, mkpts1_coords, method=cv2.RANSAC,
                                                     ransacReprojThreshold=5.0) # RANSAC threshold (tune?)

                    # Decompose M and check RANSAC inliers
                    pred_angle = None
                    trans_px_tl = None
                    estimation_succeeded = False # Flag for success
                    if M is not None:
                        num_inliers = np.sum(mask) if mask is not None else 0
                        if num_inliers < min_req_points:
                            print(f"  Skipping {filename}: RANSAC found only {num_inliers} inliers (less than {min_req_points}).")
                            # M = None # Keep M as None, estimation failed
                        else:
                            # RANSAC succeeded with enough inliers
                            estimation_succeeded = True
                            tx = M[0, 2]
                            ty = M[1, 2]
                            trans_px_tl = (tx, ty)
                            angle_rad = math.atan2(M[1, 0], M[0, 0])
                            angle_deg_raw = np.degrees(angle_rad)
                            pred_angle = angle_deg_raw % 360
                            if pred_angle < 0:
                                pred_angle += 360
                    else:
                        print(f"  Skipping {filename}: cv2.estimateAffine2D failed (returned None).")
                        # M remains None, estimation failed

                    # If estimation failed (M is None or not enough inliers), skip pair
                    if not estimation_succeeded:
                        continue

                    # Convert Coordinates (only if estimation succeeded)
                    pred_x, pred_y = convert_to_relative_center_coords(
                        trans_px_tl, pred_angle, (query_h, query_w), (ref_h, ref_w)
                    )
                    # --- End Core Prediction Logic ---

                    # If we reach here, the pair was processed successfully
                    pair_end_time = time.time()
                    prediction_times.append(pair_end_time - pair_start_time)
                    processed_count += 1

                except Exception as e:
                    # Print detailed error for the specific pair processing failure
                    print(f"  ERROR processing {filename} during timing loop:")
                    print(traceback.format_exc())
                    # Exclude pairs that errored from average time calculation

            end_time = time.time() # Overall wall time end

            # 4. Calculate and report times
            total_wall_time = end_time - start_time
            if processed_count > 0:
                 total_prediction_time = sum(prediction_times)
                 average_time = total_prediction_time / processed_count
                 print(f"\n--- Timing Results ---")
                 print(f"Successfully processed {processed_count} out of {num_to_process} selected pairs.")
                 print(f"Total prediction time (sum of successful pairs): {total_prediction_time:.3f} seconds")
                 print(f"Average prediction time per successful pair:   {average_time:.3f} seconds")
                 print(f"Total wall time for {num_to_process} attempts: {total_wall_time:.3f} seconds") # Optional
            else:
                 print("\nNo pairs were successfully processed during timing.")
                 print(f"Total wall time for {num_to_process} attempts: {total_wall_time:.3f} seconds") # Optional


    except FileNotFoundError as fnf:
        print(f"ERROR during file operations: {fnf}")
    except NameError as ne:
         # This check should have caught missing variables earlier
         print(f"UNEXPECTED ERROR: A required variable is not defined ({ne}).")
    except Exception as e:
        print(f"An unexpected error occurred OUTSIDE the timing loop:")
        print(traceback.format_exc())


print("\n--- End of Cell 10 execution ---")


--- Timing Prediction for 5 Random Image Pairs ---

--- DEBUG: Pre-requisite checks passed. Proceeding with timing logic ---
Selected 5 files for timing:
  - (1721,12857)_(210.69,-113.71)_304.59_mumbai2.png
  - (5374,2147)_(-228.25,-98.42)_276.33_mumbai2.png
  - (6402,4204)_(56.41,119.81)_320.23_mumbai.png
  - (4025,3684)_(-29.19,-116.47)_209.03_delhi.png
  - (8005,15921)_(-54.09,26.92)_50.90_mumbai2.png

--- Timing Results ---
Successfully processed 5 out of 5 selected pairs.
Total prediction time (sum of successful pairs): 1.408 seconds
Average prediction time per successful pair:   0.282 seconds
Total wall time for 5 attempts: 1.409 seconds

--- End of Cell 10 execution ---
